In [ ]:
# импортируем библиотеки и их компоненты
from yargy.interpretation import fact
from yargy.pipelines import morph_pipeline
from yargy import rule, or_, Parser
from yargy.predicates import gram, eq
import pandas as pd

# Создаем структуры для правил
Hello = fact('Hello', ['line'])
Intro = fact('Intro', ['name'])
Company = fact('Company', ['name'])
Goodbye = fact('Goodbye', ['line'])

'''
Создаем правил парсинга для извлечения данных:

HELLO - извлечение реплик приветствия менеджера
INTRO - извлечение представления менеджера и имени менеджера
COMPANY - извлечение названия компании
GOODBYE - извлечение реплик прощания менеджера
'''
hello_1 = morph_pipeline(['здравствуйте']).interpretation(Hello.line)
hello_2 = rule(morph_pipeline(['добрый']), morph_pipeline(['утро', 'день', 'вечер'])).interpretation(Hello.line)
HELLO = rule(or_(hello_1, hello_2)).interpretation(Hello)

NAME = gram('Name').interpretation(Intro.name)
part_1 = morph_pipeline(['я'])
part_2 = morph_pipeline(['звать'])
intro_1 = rule(morph_pipeline(['я']), NAME.optional(), morph_pipeline(['звать']), NAME.optional())
intro_2 = rule(morph_pipeline(['это']), NAME)
intro_3 = rule(morph_pipeline(['позволить', 'разрешить', 'представиться']).repeatable(), NAME.optional())
INTRO = rule(or_(intro_1, intro_2, intro_3)).interpretation(Intro)

NOUN = gram('NOUN')
ADJF = gram('ADJF')
COMPANY_NAME = rule(ADJF.optional().repeatable(max=3), NOUN).interpretation(Company.name)
COMPANY = rule(morph_pipeline(['компания']), COMPANY_NAME).interpretation(Company)

goodbye_1 = morph_pipeline(['до свидания']).interpretation(Goodbye.line)
goodbye_2 = rule(eq('всего'), morph_pipeline(['хорошего', 'доброго'])).interpretation(Goodbye.line)
goodbye_3 = rule(morph_pipeline(['хорошего']), morph_pipeline(['дня', 'вечера'])).interpretation(Goodbye.line)
GOODBYE = rule(or_(goodbye_1, goodbye_2, goodbye_3)).interpretation(Goodbye)


def text_parsing(arg_rule, lines):
    '''
    Применяет правило к набору реплик

    Параметры:
        arg_rule(yargy.rule):правило
        lines(list):реплики

    Возвращает:
        values(list):структуры yargy.Fact
        text(list):савпадения в текте реплик
    '''
    parser = Parser(arg_rule)
    values = []
    text = []
    for line in lines:
        matches = parser.findall(line)
        if matches:
            facts = [_.fact for _ in matches]
            for match_fact in facts:
                values += [match_fact]
        if matches:
            matches = parser.findall(line)
            for match in matches:
                for token in match.tokens:
                    text.append(token.value)
    return values, text


def test_task_parsing(df, *arg_rules):
    '''
    Применяет правило к набору реплик

    Параметры:
        df(pandas.DataFrame):набор данных
        arg_rules(list):правила
        
    Возвращает:
        df_dict(dict):результаты контроля менеджеров
    '''
    keys = ['Dialog_ID', 'Greeting_lines', 'Introduction_line', 'MNGR_name', 'CO_name', 'Farewell_lines', 'Requirement']
    mngr_lists = []
    for dlg_id in range(df['dlg_id'].nunique()):
        dlg_mngr_lines = list(df.loc[lambda df: (df['dlg_id'] == dlg_id) & (df['role'] == 'manager'), 'text'])

        hello_parse = text_parsing(HELLO, dlg_mngr_lines[:5])[0]  # проверяем первые n=5 примеров, так как приветствие должно быть
                                                                  # в начале диалога. Позволяет сэкономить время обработки текста.
                                                                  # Подобным образом ограничиваем выборку для остльных правил
        if len(hello_parse) > 0:
            hello = ' | '.join([_.line[0].upper() + _.line[1:] for _ in hello_parse])
        else:
            hello = "Менеджер не поздоровался с клиентом"

        intro_parse = text_parsing(INTRO, dlg_mngr_lines[:5])
        if len(intro_parse[0]) > 0:
            mngr_name = ''.join([_.name[0].upper() + _.name[1:] for _ in intro_parse[0]])
        else:
            mngr_name = "Менеджер не представился"

        if len(intro_parse[1]) > 0:
            intro = intro_parse[1]
            intro[0] = intro[0][0].upper() + intro[0][1:]
            for i in range(len(intro)):
                if intro[i].lower() == mngr_name.lower():
                    intro[i] = intro[i][0].upper() + intro[i][1:]
            intro = ' '.join(intro)
        else:
            intro = "Менеджер не представился"

        co_parse = text_parsing(COMPANY, dlg_mngr_lines[:5])[0]
        if len(co_parse) > 0:
            co_name = ''.join([_.name[0].upper() + _.name[1:] for _ in co_parse])
        else:
            co_name = "Менеджер не произнес название компании"

        goodbye_parse = text_parsing(GOODBYE, dlg_mngr_lines[-5:])[0]
        if len(goodbye_parse) > 0:
            goodbye = ' | '.join([_.line[0].upper() + _.line[1:] for _ in goodbye_parse])
        else:
            goodbye = "Менеджер не попращался с клиентом"

        if (hello != "Менеджер не поздоровался с клиентом") & (goodbye != "Менеджер не попращался с клиентом"):
            check = True
        else:
            check = False

        mngr_lists.append([dlg_id, hello, intro, mngr_name, co_name, goodbye, check])

    df_dict = {}
    for key in range(len(keys)):
        key_list = []
        for mngr_list in mngr_lists:
            key_list.append(mngr_list[key])
        df_dict[keys[key]] = key_list

    return df_dict

# загружаем локальную копию файла test_data.csv
df = pd.read_csv('test_data.csv')

# производим извлечение необходимой информации
test_task_dict = test_task_parsing(df, rules)

# представляем данные в виде объекта pandas.DataFrame, для удобства визуализации
test_task_df = pd.DataFrame.from_dict(test_task_dict)

# выводим результаты на экран
test_task_df

,Dialog_ID,Greeting_lines,Introduction_line,MNGR_name,CO_name,Farewell_lines,Requirement
0,0,Здравствуйте,Меня зовут Ангелина,Ангелина,Диджитал бизнес,До свидания,True
1,1,Здравствуйте,Меня зовут Ангелина,Ангелина,Диджитал бизнес,Всего хорошего | До свидания,True
2,2,Здравствуйте,Меня зовут Ангелина,Ангелина,Диджитал бизнес,Менеджер не попращался с клиентом,False
3,3,Добрый день,Меня Максим зовут,Максим,Китобизнес,Всего доброго,True
4,4,Менеджер не поздоровался с клиентом,Менеджер не представился,Менеджер не представился,Менеджер не произнес название компании,До свидания,False
5,5,Менеджер не поздоровался с клиентом,Это Анастасия,Анастасия,Менеджер не произнес название компании,До свидания | Хорошего вечера,False
